In [1]:
import numpy as np
import pandas as pd
from multiprocessing import Pool
from stop_words import get_stop_words
import inscriptis
import os
import requests
import pymorphy2
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from string import punctuation
import time
from threading import Thread
import threading
import sys
import os.path
from multiprocessing.pool import ThreadPool
from collections import defaultdict
tnz = RegexpTokenizer('\w+|[^\w\s]+')
morph = pymorphy2.MorphAnalyzer()
from layout import switch_layout
import scipy
import binascii
from sklearn.metrics import f1_score
import numpy as np
import base64
from sklearn.linear_model import LogisticRegression
import csv
from nltk.tokenize import RegexpTokenizer
import gzip
import pymorphy2
import zlib
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import importlib
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import requests as req
import zipfile
import gzip
import string
import pickle
import re
import sys
from boilerpy3 import extractors
extractor = extractors.ArticleExtractor()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vkrin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
with open('title.txt', errors='ignore', encoding='utf-8') as f:
    for line in f:
        data = line.split('\t', 1)
        if len(data)>1:
            out = open("TextRelevance/Data/"+data[0].strip()+".txt", 'a', errors='ignore')
            in_=open("dump_kr/"+data[0]+'.txt','r', encoding='utf-8', errors='ignore')
            content=in_.read()
            out.writelines(data[1].strip()+'\n')
            out.writelines(content)
            in_.close()
            out.close()
        else: 
            out = open("TextRelevance/Data/"+data[0].strip()+".txt", 'a', errors='ignore')
            in_=open("dump_kr/"+data[0]+'.txt','r', encoding='utf-8', errors='ignore')
            content=in_.read()
            out.writelines(content)
            in_.close()
            out.close()

In [2]:
TRACE_NUM = 100
import logging
importlib.reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

def trace(items_num, trace_num=TRACE_NUM):
    if items_num % trace_num == 0: logging.info("Complete items %05d" % items_num)

In [1]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]'] or isinstance(element, Comment):
        return False
    else:
        return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    
    links = []
    links_t=''
    for link in soup.findAll('a', attrs={'href': re.compile("^http://")}):
        links.append(link.get('href'))
        links_t+=(' '+link.get_text())
    
    return ' '.join(visible_texts), links, links_t

def tokenize_me(file_text):
    tokens = nltk.word_tokenize(file_text)
    tokens = [morph.parse(i.lower())[0].normal_form for i in tokens if ( i not in string.punctuation )]
 
    stop_words = stopwords.words('russian')+stopwords.words('english')
    stop_words.extend(map(lambda x: x, 
                          ['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', 'к', 'на']))
    tokens = [i for i in tokens if ( i not in stop_words )]     
    return ' '.join(tokens)

In [38]:
def html2text_bs(raw_html):
    from bs4 import BeautifulSoup
    """
    Тут производится извлечения из html текста
    """
    soup = BeautifulSoup(raw_html, "html")
    [s.extract() for s in soup(['script', 'style'])]
    return soup.get_text()

def html2text_title(raw_html):
    from bs4 import BeautifulSoup
    """
    Тут производится извлечения title
    """
    titles=''
    soup = BeautifulSoup(raw_html, "html")
    s=soup.find_all('title')
    if s:
        for st in s:
            titles+=' '+(st.get_text()).strip()
        return titles.lower().strip()
    return ''

def html2text_ss(raw_html):
    from bs4 import BeautifulSoup
    """
    Тут производится извлечения ссылок и анкоров
    """
    soup = BeautifulSoup(raw_html, "html")
    links=[]
    texts=''
    for link in soup.find_all('a', href=True):
        links.append(link['href'])
        for text in link.get_text().split():
            texts=texts+' '+text
    return ' '.join(links),' '.join(tokenize_me(texts))

def html2text_bs_visible(content):
    from bs4 import BeautifulSoup
    """
    Тут производится извлечения из html текста, который видим пользователю
    """
    soup = BeautifulSoup(content, "html")    
    [s.extract() for s in soup(['style', 'script', '[document]', 'head', 'title'])]
    return norm((soup.get_text()))

def html2text_boilerpipe(content):
    doc = extractor.get_content(content)
    return norm(doc)


In [5]:
folders={'20170702':4532+1, '20170704': 4616+1, '20170707': 4609+1, '20170708': 4666+1, '20170710': 4803+1, '20170711': 4877+1, '20170717': 4954+1, '20170726': 5049+1}

In [6]:
exc={'20170702':[], '20170704': [], '20170707': [], '20170708': [], '20170710': [], '20170711': [], '20170717': [], '20170726': []}

In [7]:
exc2={'20170702':[], '20170704': [], '20170707': [], '20170708': [], '20170710': [], '20170711': [], '20170717': [], '20170726': []}

In [8]:
urls=pd.read_csv('urls.numerate.csv',header=None, sep='\t')

In [9]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [10]:
_eng_chars = u"~!@#$%^&qwertyuiop[]asdfghjkl;'zxcvbnm,./QWERTYUIOP{}ASDFGHJKL:\"|ZXCVBNM<>?"
_rus_chars = u"ё!\"№;%:?йцукенгшщзхъфывапролджэячсмитьбю.ЙЦУКЕНГШЩЗХЪФЫВАПРОЛДЖЭ/ЯЧСМИТЬБЮ,"
_trans_table = dict(zip(_eng_chars, _rus_chars))
 
def fix_layout(s):
    return u''.join([_trans_table.get(c, c) for c in s])

In [29]:
urls_dict=dict()
for i,k in zip(urls[0],urls[1]):
    urls_dict[k]=i

{'eva.ru/forum/topic-tree-mobile.htm?messageId=89211158&topicId=3285169': 1,
 'e1.ru/articles/travel/page_4/009/994/article_9994.html': 2,
 'bikepost.ru/qa/post/9065/Kak-pravilno-sdelat-vizu-v-Polshu-shengen.html': 3,
 'rutax.polpred.com/?ns=1&page=2': 4,
 'coollib.com/b/259649/read': 5,
 'coollib.com/b/244202/read': 6,
 'rabotatut.do.am/index/o_vizakh/0-5': 7,
 'kaliningrad.zoon.ru/hotels/turisticheskoe_agentstvo_dyuna39/social': 8,
 'forum.materinstvo.ru/lofiversion/index.php/t1358686': 9,
 'bookslist.me/read2.php?id=8834': 10,
 'lifecz.ru/threads/1375/page-111': 11,
 'studenttests.ru/%D1%83%D0%BD%D0%B8%D0%B2%D0%B5%D1%80%D1%81%D0%B8%D1%82%D0%B5%D1%82%D1%8B/Niderlandy': 12,
 'gulfcoolcontracting.com/uploads/userfiles/file/dom8197-sertifikat-na-pokupku-bileta-v-teatr-jam7575.xml': 13,
 'digesttour.ru/node/749': 14,
 'forum.immigration.cz/archive/index.php/t-5389.html': 15,
 'spaininvestor.com/Views/TextosWebForm.aspx?IdTexto=379': 16,
 'sarov24.ru/mneniya/640-bezvizovye-strany-2017-dly

In [12]:
import shutil
import sys
querys = {}
with open('title.txt',encoding='utf-8') as f:
    for num_line, line in enumerate(f):
        data = line.split('\t', 1)
        try:
            if len(data)>1:
                querys[int(data[0])]=(data[1])
            else: 
                querys[int(data[0])]=" "
        except:
            continue
print (len(querys))

38114


In [13]:
for i,k in urls_dict.items():
    if k not in querys.keys():
        print(k)

In [14]:
def extracturl(text):
    first = text.find('http')
    if first == -1:
        return None
    if text[0] == 's':
        first+=8
    else:
        first+=7
    text=text[first:]
    fslash = text.find('/')
    text = text[:fslash]
    dot = text.rfind('.')
    text = text[:dot]
    lst = text.split('.')
    strk = ' '.join(lst)
    return strk

In [15]:
def norm(text):
    restxt=' '
    text=text.lower()
    for token in tnz.tokenize(text):
        if (token not in stopwords.words('Russian')) and (token != '©') and (token.isdigit() == False) \
        and isnotpunct(token) and (token not in stopwords.words('English')) and (token != '…') and \
        (token != '»') and (token != '«') and (token != '—') and hasnotdigits(token):
            token = morph.parse(token)[0].normal_form
            if isRussianWord(token) or isEnglishWord(token):
                if token in '.?!':
                    restxt+=token
                else:
                    restxt+=" "+token   
    return restxt  
def isnotpunct(strk):
    for s in strk:
        if s in '"#$%&\'()*+,-/:;<=>@[\\]^_`{|}~':
            return False
    return True  

def hasnotdigits(token):
    for s in token:
        if s in '0123456789':
            return False
    return True

def isRussianWord(w):
    alphabet = "абвгдеёжзийклмнопрстуфхцчшщъыьэюя"+'.?!'
    flag=True
    for c in w:
        if(not c in alphabet):
            flag=False
            break
    return flag

def isEnglishWord(w):
    alphabet = "qwertyuiopasdfghjklzxcvbnm"+'?.!'
    flag=True
    for c in w:
        if(not c in alphabet):
            flag=False
            break
    return flag

In [39]:
def func_wrapper(k):
    out_stem = open("queryies_not_normal.txt", 'a', errors='ignore')
    for num in range(folders[k]):
        try:
            f = open('content/'+k+'/doc.'+str(num).zfill(4)+'.dat', encoding='utf-8', errors='ignore')
            url = f.readline().strip()
            content=f.read()
            f.close()
            text=html2text_title(content)
        except Exception as ex:
            print(k,num,ex)
            f.close()
            continue
        out_stem.write(str(urls_dict[url])+'\t'+text+'\n')
    out_stem.close()
    return

In [20]:
f = open('content/'+'20170702'+'/doc.'+str(0).zfill(4)+'.dat', encoding='utf-8', errors='ignore')
url = f.readline().strip()
content=f.read()
f.close()

In [21]:
html2text_title(content)

'Манты с тыквой пошаговый фото-рецепт'

In [34]:
doc=extractor.get_doc(content)

In [35]:
doc.content

'» Манты с тыквой пошаговый фото-рецепт\nМанты с тыквой пошаговый фото-рецепт\nкомментарии 0\nГотовим манты с тыквой. Манты с тыквой — традиционное узбекское блюдо с прекрасными вкусовыми качествами, которые наверняка оценят любители тыквы. К тому же, тыква приготовленная на пару очень полезна. В классическом рецепте используется курдючное сало — продукт довольно редкий, поэтому я заменила его на сливочное масло, что вполне допустимо и не менее вкусно. Так же я позволила себе некоторую вольность по отношению к специям и взяла свою любимую, именно в сочетании с тыквой, совсем не узбекскую смесь «хмели-сунели», с ней получается очень вкусно. Хорошо сюда подходит и порошок карри.\nВ качестве соуса к этому блюду отлично подойдут кисломолочные продукты, смешанные с солью, кумином и зеленью: катык, йогурт или сметана.\nИнгредиенты:\nПерец чёрный молотый 1 щепотка\nМасло сливочное 70 г\nПодготовьте основные ингредиенты: тесто, тыкву, масло, специи, лук и чеснок.\nШаг 2\nТесто можно приготовит

In [37]:
html2text_boilerpipe(content)

'» мант тыква пошаговый фото-рецепт мант тыква пошаговый фото-рецепт комментарий 0 готовить мант тыква мант тыква традиционный узбекский блюдо прекрасный вкусовой качество который наверняка оценить любитель тыква тыква приготовить пара очень полезный классический рецепт использоваться курдючный сало продукт довольно редкий поэтому заменить сливочный масло вполне допустимый менее вкусно позволить некоторый вольность отношение специя взять свой любимый именно сочетание тыква узбекский смесь « хмель-сунель » получаться очень вкусно сюда подходить порошок карри качество соус блюдо отлично подойти кисломолочный продукт смешанный соль кумином зелень катык йогурт сметана ингредиент перец чёрный молотый 1 щепотка масло сливочный 70 год подготовить основной ингредиент тесто тыква масло специя лука чеснок шаг 2 тесто приготовить заранее просеять горка мука разделочный доска сделать углубление посредине вбить туда яйцо аккуратно влить тепло вода добавить соль шаг 3 начинать замешивать тесто снача

In [18]:
extractor = extractors.ArticleExtractor()
content = extractor.get_content(content)

In [15]:
(tokenize_me(content.lower().replace('\n', ' ').strip())).replace(' .','.')

'мант тыква пошаговый фото-рецепт загрузка пожалуйста подождать... главный сайт контакт статистика рекламодатель подписаться rss главный мультиварка весь рецепт brand 502 moulinex panasonic sr polaris pmc redmond rmc vitek vt первое борщ бульон рассольник суп суп-пюре холодное суп щи разный блюдо птица мясной блюдо рыбный блюдо овощной блюдо пельмень т.п шашлык каша разный салат мясной салат рыбный салат овощной салат мясо птица бобовый закуска бутерброд мясной закуска рыбный закуска овощ мясо птица сыр яйцо соус горячее мясной рыбный грибной холодное заправка разный десерт десерт торт печение мусса желе суфле пудинг мороженое зефира разный мучной блин оладья выпечка пирог пирожок пицца напиток смузи игристый фрукт десертный крюшон глинтвейн молочный чаять кофе разный раздел консервация пост строгий пост строгий детский меню дар море блюдо яйцо кухня мир диета пряность совет разный povarito.ru » второе блюдо » мант тыква пошаговый фото-рецепт мант тыква пошаговый фото-рецепт написать p

In [40]:
for i in list(folders.keys()):
    print(i)
    func_wrapper(i)

20170702
20170704
20170707
20170708
20170708 3477 [Errno 2] No such file or directory: 'content/20170708/doc.3477.dat'
20170710
20170711
20170717
20170717 1615 [Errno 2] No such file or directory: 'content/20170717/doc.1615.dat'
20170726


In [13]:
querys=[]
out_stem = open("querys.txt", 'w', encoding='utf-8')
with open('queries.numerate.csv',encoding='utf-8') as f:
    for num_line, line in enumerate(f):
        data=line.strip().split('\t', 1)
        query=norm(data[1]).strip()
        if query=='':
            out_stem.writelines(str(data[0])+','+data[1]+'\n')
        else:
            out_stem.writelines(str(data[0])+','+query+'\n')
    out_stem.close()

In [ ]:
tim = time.time()
with ThreadPool(8) as pool:
    pool.map(func_wrapper, list(folders.keys()))
print('overall time',time.time()-tim,'seconds')

19:52:19 ERROR:Error parsing HTML
Traceback (most recent call last):
  File "C:\Users\vkrin\Anaconda3\lib\site-packages\boilerpy3\extractors.py", line 81, in parse_doc
    bp_parser.feed(input_str)
  File "C:\Users\vkrin\Anaconda3\lib\site-packages\boilerpy3\parser.py", line 651, in feed
    HTMLParser.feed(self, data)
  File "C:\Users\vkrin\Anaconda3\lib\html\parser.py", line 111, in feed
    self.goahead(0)
  File "C:\Users\vkrin\Anaconda3\lib\html\parser.py", line 173, in goahead
    k = self.parse_endtag(i)
  File "C:\Users\vkrin\Anaconda3\lib\html\parser.py", line 421, in parse_endtag
    self.handle_endtag(elem)
  File "C:\Users\vkrin\Anaconda3\lib\site-packages\boilerpy3\parser.py", line 658, in handle_endtag
    self.end_element(tag)
  File "C:\Users\vkrin\Anaconda3\lib\site-packages\boilerpy3\parser.py", line 492, in end_element
    self.label_stacks.pop()
IndexError: pop from empty list

During handling of the above exception, another exception occurred:

Traceback (most rece

20170704 0 'NoneType' object has no attribute 'text_blocks'


19:52:20 ERROR:Error parsing HTML
Traceback (most recent call last):
  File "C:\Users\vkrin\Anaconda3\lib\site-packages\boilerpy3\extractors.py", line 81, in parse_doc
    bp_parser.feed(input_str)
  File "C:\Users\vkrin\Anaconda3\lib\site-packages\boilerpy3\parser.py", line 651, in feed
    HTMLParser.feed(self, data)
  File "C:\Users\vkrin\Anaconda3\lib\html\parser.py", line 111, in feed
    self.goahead(0)
  File "C:\Users\vkrin\Anaconda3\lib\html\parser.py", line 173, in goahead
    k = self.parse_endtag(i)
  File "C:\Users\vkrin\Anaconda3\lib\html\parser.py", line 421, in parse_endtag
    self.handle_endtag(elem)
  File "C:\Users\vkrin\Anaconda3\lib\site-packages\boilerpy3\parser.py", line 658, in handle_endtag
    self.end_element(tag)
  File "C:\Users\vkrin\Anaconda3\lib\site-packages\boilerpy3\parser.py", line 492, in end_element
    self.label_stacks.pop()
IndexError: pop from empty list

During handling of the above exception, another exception occurred:

Traceback (most rece

20170704 1 'NoneType' object has no attribute 'text_blocks'


19:52:24 ERROR:Error parsing HTML
Traceback (most recent call last):
  File "C:\Users\vkrin\Anaconda3\lib\site-packages\boilerpy3\extractors.py", line 81, in parse_doc
    bp_parser.feed(input_str)
  File "C:\Users\vkrin\Anaconda3\lib\site-packages\boilerpy3\parser.py", line 651, in feed
    HTMLParser.feed(self, data)
  File "C:\Users\vkrin\Anaconda3\lib\html\parser.py", line 111, in feed
    self.goahead(0)
  File "C:\Users\vkrin\Anaconda3\lib\html\parser.py", line 173, in goahead
    k = self.parse_endtag(i)
  File "C:\Users\vkrin\Anaconda3\lib\html\parser.py", line 421, in parse_endtag
    self.handle_endtag(elem)
  File "C:\Users\vkrin\Anaconda3\lib\site-packages\boilerpy3\parser.py", line 658, in handle_endtag
    self.end_element(tag)
  File "C:\Users\vkrin\Anaconda3\lib\site-packages\boilerpy3\parser.py", line 492, in end_element
    self.label_stacks.pop()
IndexError: pop from empty list

During handling of the above exception, another exception occurred:

Traceback (most rece

...

20170717 314 'NoneType' object has no attribute 'text_blocks'


20:24:29 ERROR:Error parsing HTML
Traceback (most recent call last):
  File "C:\Users\vkrin\Anaconda3\lib\site-packages\boilerpy3\extractors.py", line 81, in parse_doc
    bp_parser.feed(input_str)
  File "C:\Users\vkrin\Anaconda3\lib\site-packages\boilerpy3\parser.py", line 651, in feed
    HTMLParser.feed(self, data)
  File "C:\Users\vkrin\Anaconda3\lib\html\parser.py", line 111, in feed
    self.goahead(0)
  File "C:\Users\vkrin\Anaconda3\lib\html\parser.py", line 173, in goahead
    k = self.parse_endtag(i)
  File "C:\Users\vkrin\Anaconda3\lib\html\parser.py", line 421, in parse_endtag
    self.handle_endtag(elem)
  File "C:\Users\vkrin\Anaconda3\lib\site-packages\boilerpy3\parser.py", line 658, in handle_endtag
    self.end_element(tag)
  File "C:\Users\vkrin\Anaconda3\lib\site-packages\boilerpy3\parser.py", line 492, in end_element
    self.label_stacks.pop()
IndexError: pop from empty list

During handling of the above exception, another exception occurred:

Traceback (most rece

20170707 281 'NoneType' object has no attribute 'text_blocks'


20:24:30 WARNING:Warning: SAX input contains nested A elements -- You have probably hit a bug in your HTML parser (e.g., NekoHTML bug #2909310). Please clean the HTML externally and feed it to BoilerPy3 again. Trying to recover somehow...
20:24:31 ERROR:Error parsing HTML
Traceback (most recent call last):
  File "C:\Users\vkrin\Anaconda3\lib\site-packages\boilerpy3\extractors.py", line 81, in parse_doc
    bp_parser.feed(input_str)
  File "C:\Users\vkrin\Anaconda3\lib\site-packages\boilerpy3\parser.py", line 651, in feed
    HTMLParser.feed(self, data)
  File "C:\Users\vkrin\Anaconda3\lib\html\parser.py", line 111, in feed
    self.goahead(0)
  File "C:\Users\vkrin\Anaconda3\lib\html\parser.py", line 173, in goahead
    k = self.parse_endtag(i)
  File "C:\Users\vkrin\Anaconda3\lib\html\parser.py", line 421, in parse_endtag
    self.handle_endtag(elem)
  File "C:\Users\vkrin\Anaconda3\lib\site-packages\boilerpy3\parser.py", line 658, in handle_endtag
    self.end_element(tag)
  File "C

20170704 333 'NoneType' object has no attribute 'text_blocks'
20170707 284 'NoneType' object has no attribute 'text_blocks'


In [48]:
f = open('content/'+'20170710'+'/doc.'+str(594).zfill(4)+'.dat',encoding='utf-8', errors='ignore')
url = f.readline().strip()
content=f.read()
f.close()
text=inscriptis.get_text(content)
norm(text)

'  поиск искать заголовок сообщение пользователь имя участник разделять запятая новый искать тема искать раздел отображать результат вид быстрый поиск последний сообщение ... форум дом стройка перекрытие. дорогой форумчанин! напоминать самый крутой проект история forumhouse конкурс дом год продолжаться! конкурс принять участие семья любой регион россия. выполнять задание любой порядок... следить стройка раздел история скрыть объявление оценка перекрытие метр деревянный балка тема раздел перекрытие создать пользователь строюсамд... опция тема опция тема страница вперёд. регистрация.. сообщение благодарность строюсамд участник строюсамд участник регистрация.. сообщение благодарность адрес верхний тур перекрытие метр деревянный балка комната планировать перекрыть брус сшить шпилька получиться шаг быть делать блок сегодня попробовать сшить брус поставить литровый бочка вода прогиб составить мм. прогиб маленький настелить половина? строюсамд.. цитировать. регистрация.. сообщение благодарнос

In [12]:
x=0
for i in exc.keys():
    x=x+len(exc[i])
x

30